In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Giới thiệu
Đây là cuộc thi Plant Pathology 2021. Mục tiêu của cuộc thi này là xác định bệnh của cây dựa trên hình ảnh của lá, có rất nhiều loại bệnh. Giải quyết vấn đề này rất quan trọng vì chẩn đoán bệnh cây sớm có thể tiết kiệm hàng tấn nông sản hàng năm. 
Trong notebook này, chúng ta sẽ khám phá dữ liệu này có những gì, trực quan hóa nó thông qua các hình ảnh, biểu đồ

**Xác định mục tiêu:**

Mục tiêu chính của cuộc thi là phát triển các mô hình dựa trên máy học để phân loại chính xác một hình ảnh lá nhất định từ bộ dữ liệu thử nghiệm cho một loại bệnh cụ thể và xác định một bệnh riêng lẻ từ nhiều triệu chứng bệnh trên một hình ảnh lá đơn

**Mô tả dữ liệu:**

Dữ liệu lưu giữ hình ảnh của cây táo. Lá cây khỏe mạnh và bị nhiễm bệnh.

Files train.csv - dữ liệu tập huấn luyện.

Image - ID của hình ảnh

Label - các lớp mục tiêu thể hiện tất cả các bệnh được tìm thấy trong hình ảnh. Những lá không tốt có quá nhiều bệnh để phân loại bằng mắt thường sẽ có lớp phức tạp, và cũng có thể có một tập hợp con của các bệnh được xác định.

sample_submission.csv - Tệp gửi mẫu ở định dạng:

1. image
2. labels

train_images - tập ảnh train.

test_images - tập ảnh test.

# Cấu hình

In [ ]:
import torch
import cv2
import os
import torch.nn as tnn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns

from PIL import Image
from skimage import io, transform
from torchvision.transforms import transforms
from torchvision import utils
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter


# Đường dẫn 

In [ ]:
IMAGE_PATH = "../input/plant-pathology-2021-fgvc8/train_images/"
TEST_IMG_PATH = "../input/plant-pathology-2021-fgvc8/test_images/"
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train.csv"
SUB_PATH = "../input/plant-pathology-2021-fgvc8/sample_submission.csv"

# Dữ liệu

In [ ]:
train_labels = pd.read_csv(TRAIN_PATH)
train_labels

In [ ]:
train_labels['labels'].unique()

#### **Nhận xét:** 
Dữ liệu gồm 12 loại nhãn. Trong đó có 6 nhãn chính là  healthy,scab, rust, complex, powdery_mildew và frog_eye_leaf_spot, 6 nhãn còn lại là các nhãn kết hợp từ 6 nhãn chính

# Thống kê số lượng nhãn

In [ ]:
plt.figure(figsize=(18,12))
plt.title("Phân phối số lượng ảnh trong các nhãn",size= 25)
plt.ylabel("Số lượng ảnh", size=20);
plt.xlabel("Nhãn", size=20);
labels = sns.barplot(train_labels.labels.value_counts().index,train_labels.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)
plt.savefig('plot.png')

### Nhận xét:
Các bệnh đơn lẻ chiếm tỉ lệ lớn trong khi các bệnh kết hợp rất hiếm.

Khoảng 51% dữ liệu đầu vào thuộc loại Scab hoặc Healthy. 

==> Do đó ta chuyển từ bài toán phân loại một nhãn duy nhất cho ảnh,sang bài toán phân lớp đa nhãn

In [ ]:
mlb = MultiLabelBinarizer().fit(train_labels.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mlb.transform(train_labels.labels.apply(lambda x : x.split())), columns = mlb.classes_)

labels = pd.concat([train_labels['image'], labels], axis=1)
labels.head()

# Thống kê số lượng ảnh đa nhãn

In [ ]:
data = ['1','2','3']
value = labels.iloc[:,1:].sum(axis=1).value_counts().values
colors = ['mediumturquoise', 'burlywood','sandybrown']
plt.figure(figsize=(8, 8))
plt.bar(data, value, color = colors)
plt.title('Ảnh có nhiều nhãn',fontsize = 14)
plt.xlabel('Số nhãn',fontsize = 12)
plt.ylabel('Số lượng ảnh',fontsize = 12)
plt.savefig('plot2.png')
plt.show()

**Nhận xét:**
Tập dữ liệu bị mất cân bằng khá nhiều

# Kích thước của ảnh

In [ ]:
img_name = labels.iloc[:,0].tolist()

In [ ]:
hs = []
ws = []
for i in range(len(img_name)):
        img = Image.open(IMAGE_PATH+(img_name[i]))
        h, w = img.size
        hs.append(h)
        ws.append(w)

## Chiều cao của ảnh

In [ ]:
labels, values = zip(*Counter(hs).items())

indexes = np.arange(len(labels))
width = 1

plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels)
plt.savefig('plot4.png')
plt.show()

## Chiều rộng của ảnh

In [ ]:
labels, values = zip(*Counter(ws).items())

indexes = np.arange(len(labels))
width = 1

plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels)
plt.savefig('plot5.png')
plt.show()

### Nhận xét:
Các ảnh có kích cỡ không đồng đều, bên cạnh đó dữ liệu ảnh rất lớn

==> Giải pháp: Giảm kích cỡ ảnh, đưa dữ liệu về kích cỡ đồng nhất

# Dữ liệu theo từng nhãn

In [ ]:
labels = pd.DataFrame(mlb.transform(train_labels.labels.apply(lambda x : x.split())), columns = mlb.classes_)

labels = pd.concat([train_labels['image'], labels], axis=1)

In [ ]:
def visualize_batch(path,image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(path, image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        plt.axis("off")
        plt.savefig('plot3.png')
    plt.show()

In [ ]:
img_s = train_labels.sample(9)
image_ids = img_s["image"].values
labels_s = img_s["labels"].values
visualize_batch(IMAGE_PATH,image_ids,labels_s)

In [ ]:
l_complex = labels.loc[labels['complex'] == 1].iloc[:,0].tolist()
frog_eye_leaf_spot = labels.loc[labels['frog_eye_leaf_spot'] == 1].iloc[:,0].tolist()
healthy = labels.loc[labels['healthy'] == 1].iloc[:,0].tolist()
powdery_mildew = labels.loc[labels['powdery_mildew'] == 1].iloc[:,0].tolist()
rust = labels.loc[labels['rust'] == 1].iloc[:,0].tolist()
scab = labels.loc[labels['scab'] == 1].iloc[:,0].tolist()

## Healthy

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +healthy[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Những lá khỏe mạnh là những lá xanh hoàn toàn 

## Complex

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +l_complex[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Những lá này có màu xanh nhợt nhạt, các đốm vàng nâu

## Frog Eye Leaf Spot 

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +frog_eye_leaf_spot[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Triệu chứng chẩn đoán rõ nhất của bệnh Frog Eye Leaf Spot là những đốm có góc cạnh với tâm màu xám nhạt và rìa lá màu tím đến nâu đỏ rõ rệt. Không có quầng vàng xung quanh chỗ bệnh. Các đốm lá có thể đơn lẻ hoặc hợp nhất để tạo thành các đốm lớn hơn. 

## Scab

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +scab[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Trong những hình ảnh trên, chúng ta có thể thấy những chiếc lá nhãn scab có những vết màu nâu lớn và những vết loang lổ khắp mặt lá. 

## Powdery Mildew

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +powdery_mildew[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Dấu hiệu của các lá bị Powdery Mildew được bao phủ một lớp nấm xám dày đặc như bột phấn hết cả phiến lá. Lớp phấn trắng xuất hiện trên cả thân, cành, hoa làm hoa khô rụng và chết.

## Rust

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(0,9):
        img_array = np.array(Image.open(IMAGE_PATH +rust[i]))
        fig.add_subplot(3, 3, i+1) 
        plt.imshow(img_array)

Thông qua những hình ảnh trên, những chiếc lá bị "rust" có một vài đốm màu vàng nâu trên khắp mặt lá.